# Strands Agent Content Generator - AWS Bedrock Implementation

This notebook demonstrates a complete production-ready implementation of a multi-agent content generation system using:
- **AWS Bedrock** for LLM capabilities
- **Strands Agent Framework** for orchestration
- **Step-by-step workflow** from outline to polished content

## Use Case: Technical Documentation Generator
We'll build a system that generates comprehensive technical guides on AWS services.

---

## Prerequisites & Installation

Run this cell to install all required dependencies:

In [3]:
%pip install -q boto3 strands-agents strands-agents-tools langchain langchain-aws python-dotenv


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Configuration & AWS Credentials

**Important:** Before running this cell, ensure you have AWS credentials configured.

### Option 1: Use AWS CLI (Recommended)
```bash
aws configure
```
This will create `~/.aws/credentials` file with your credentials.

### Option 2: Set Environment Variables
```bash
# Windows PowerShell
$env:AWS_ACCESS_KEY_ID="your-access-key"
$env:AWS_SECRET_ACCESS_KEY="your-secret-key"
$env:AWS_DEFAULT_REGION="us-east-1"

# Linux/Mac
export AWS_ACCESS_KEY_ID="your-access-key"
export AWS_SECRET_ACCESS_KEY="your-secret-key"
export AWS_DEFAULT_REGION="us-east-1"
```

### Required Permissions
Your AWS credentials need the following permissions:
- `bedrock:InvokeModel` - To call Bedrock models
- `bedrock:ListFoundationModels` - To list available models (optional)

**Note:** If you get "UnrecognizedClientException", your credentials may be invalid or expired. Generate new credentials from the AWS IAM console.

In [4]:
import os
import boto3
from datetime import datetime
import json
from botocore.exceptions import ClientError, NoCredentialsError

# ============================================
# AWS Configuration
# ============================================
# Credentials will be loaded from:
# 1. Environment variables (AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)
# 2. ~/.aws/credentials file (created by 'aws configure')
# 3. IAM role (if running on EC2/Lambda)
# ============================================

AWS_REGION = os.environ.get('AWS_DEFAULT_REGION', 'us-east-1')  # Change to your preferred region

# Try to get credentials from environment or default credential chain
try:
    session = boto3.Session()
    credentials = session.get_credentials()
    
    if credentials is None:
        raise NoCredentialsError("No AWS credentials found")
    
    # Use credentials from the session (supports all credential sources)
    AWS_ACCESS_KEY_ID = credentials.access_key
    AWS_SECRET_ACCESS_KEY = credentials.secret_key
    
    print("✅ AWS credentials loaded successfully!")
    print(f"  Access Key: {AWS_ACCESS_KEY_ID[:10]}...")
    print(f"  Region: {AWS_REGION}")
    
    # Initialize Bedrock client with extended timeouts for large content generation
    # Configure longer timeouts to handle large content processing
    from botocore.config import Config
    config = Config(
        read_timeout=300,  # 5 minutes for large content generation
        connect_timeout=60,  # 1 minute to establish connection
        retries={
            'max_attempts': 3,  # Retry up to 3 times
            'mode': 'adaptive'  # Adaptive retry mode
        }
    )
    
    bedrock_runtime = boto3.client(
        service_name='bedrock-runtime',
        region_name=AWS_REGION,
        config=config
    )
    
    print("✅ AWS Bedrock client initialized successfully!")
    print("  Configured with extended timeouts (5 min) for large content processing")
    
except NoCredentialsError as e:
    print("✗ ERROR: No AWS credentials found!")
    print("\n  Please configure AWS credentials using one of these methods:")
    print("  1. Set environment variables:")
    print("     export AWS_ACCESS_KEY_ID='your-access-key'")
    print("     export AWS_SECRET_ACCESS_KEY='your-secret-key'")
    print("     export AWS_DEFAULT_REGION='us-east-1'")
    print("\n  2. Use AWS CLI:")
    print("     aws configure")
    print("\n  3. Create ~/.aws/credentials file manually")
    print("\n  Then restart the kernel and run this cell again.")
    raise
    
except ClientError as e:
    error_code = e.response.get('Error', {}).get('Code', 'Unknown')
    if error_code == 'UnrecognizedClientException':
        print("✗ ERROR: Invalid AWS credentials!")
        print("  The security token included in the request is invalid.")
        print("\n  SOLUTION:")
        print("  1. Verify your credentials are correct")
        print("  2. Check if your credentials have expired")
        print("  3. Generate new credentials from AWS IAM console")
        print("  4. Update your credentials and restart the kernel")
    else:
        print(f"✗ ERROR: {error_code}")
        print(f"  {str(e)}")
    raise
    
except Exception as e:
    print(f"✗ ERROR: {type(e).__name__}: {e}")
    raise

✅ AWS credentials loaded successfully!
  Access Key: AKIAQPRFFU...
  Region: us-east-1
✅ AWS Bedrock client initialized successfully!
  Configured with extended timeouts (5 min) for large content processing


## Helper Functions & Base Classes

These provide the foundation for our Strands Agent implementation:

In [5]:
import time
from botocore.exceptions import ReadTimeoutError, ClientError

class BedrockAgent:
    """Base agent class using AWS Bedrock with retry logic"""
    
    def __init__(self, name, system_prompt, model_id="anthropic.claude-3-sonnet-20240229-v1:0"):
        self.name = name
        self.system_prompt = system_prompt
        self.model_id = model_id
        self.conversation_history = []
        
    def generate(self, user_message, temperature=0.7, max_tokens=4096, max_retries=3):
        """Generate a response using AWS Bedrock with retry logic"""
        
        # Prepare the request body
        request_body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "temperature": temperature,
            "system": self.system_prompt,
            "messages": [
                {
                    "role": "user",
                    "content": user_message
                }
            ]
        }
        
        # Retry logic with exponential backoff
        last_exception = None
        for attempt in range(max_retries):
            try:
                # Call Bedrock
                response = bedrock_runtime.invoke_model(
                    modelId=self.model_id,
                    body=json.dumps(request_body)
                )
                
                # Parse response
                response_body = json.loads(response['body'].read())
                content = response_body['content'][0]['text']
                
                # Store in conversation history
                self.conversation_history.append({
                    "user": user_message,
                    "assistant": content,
                    "timestamp": datetime.now().isoformat()
                })
                
                return content
                
            except ReadTimeoutError as e:
                last_exception = e
                if attempt < max_retries - 1:
                    wait_time = (2 ** attempt) * 5  # Exponential backoff: 5s, 10s, 20s
                    print(f"⚠️  Timeout on attempt {attempt + 1}/{max_retries}. Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    print(f"✗ Timeout after {max_retries} attempts. The content may be too large.")
                    print("  Consider breaking the content into smaller chunks.")
                    raise
                    
            except ClientError as e:
                error_code = e.response.get('Error', {}).get('Code', 'Unknown')
                if error_code in ['ThrottlingException', 'ServiceUnavailable'] and attempt < max_retries - 1:
                    wait_time = (2 ** attempt) * 5
                    print(f"⚠️  {error_code} on attempt {attempt + 1}/{max_retries}. Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                    last_exception = e
                else:
                    raise
                    
            except Exception as e:
                # For other exceptions, don't retry
                raise
        
        # If we get here, all retries failed
        if last_exception:
            raise last_exception
    
    def get_history(self):
        """Return conversation history"""
        return self.conversation_history


class OutlineEvaluator:
    """Evaluates outline quality based on criteria"""
    
    def __init__(self, criteria):
        self.criteria = criteria
        
    def evaluate(self, outline):
        """Evaluate outline and return score and feedback"""
        evaluator = BedrockAgent(
            name="OutlineEvaluator",
            system_prompt=f"""You are an expert content evaluator. 
            Evaluate the following outline based on these criteria:
            {json.dumps(self.criteria, indent=2)}
            
            Provide a score from 0.0 to 1.0 and specific feedback.
            Return your response as JSON with 'score' and 'feedback' keys."""
        )
        
        response = evaluator.generate(f"Evaluate this outline:\n\n{outline}")
        
        try:
            # Extract JSON from response
            json_start = response.find('{')
            json_end = response.rfind('}') + 1
            result = json.loads(response[json_start:json_end])
            return type('EvalResult', (), result)
        except:
            # Fallback if JSON parsing fails
            return type('EvalResult', (), {'score': 0.7, 'feedback': response})


def parse_sections(outline):
    """Parse outline into individual sections"""
    sections = []
    lines = outline.split('\n')
    
    for line in lines:
        line = line.strip()
        if line.startswith('#') or (line and line[0].isdigit() and '.' in line):
            # Remove markdown headers or numbering
            section = line.lstrip('#').lstrip('0123456789.').strip()
            if section:
                sections.append(section)
    
    return sections


def split_content_into_chunks(content, max_chunk_size=50000):
    """
    Split large content into smaller chunks to avoid timeout issues.
    Tries to split at section boundaries (## headers) when possible.
    
    Args:
        content: The content string to split
        max_chunk_size: Maximum size of each chunk in characters
    
    Returns:
        List of content chunks
    """
    if len(content) <= max_chunk_size:
        return [content]
    
    chunks = []
    lines = content.split('\n')
    current_chunk = []
    current_size = 0
    
    for line in lines:
        line_size = len(line) + 1  # +1 for newline
        
        # If adding this line would exceed the limit and we have content
        if current_size + line_size > max_chunk_size and current_chunk:
            # Save current chunk
            chunks.append('\n'.join(current_chunk))
            current_chunk = []
            current_size = 0
        
        # If a single line is too long, split it (shouldn't happen normally)
        if line_size > max_chunk_size:
            # Split the long line
            words = line.split()
            temp_line = []
            temp_size = 0
            for word in words:
                word_size = len(word) + 1
                if temp_size + word_size > max_chunk_size and temp_line:
                    chunks.append(' '.join(temp_line))
                    temp_line = []
                    temp_size = 0
                temp_line.append(word)
                temp_size += word_size
            if temp_line:
                current_chunk.append(' '.join(temp_line))
                current_size += temp_size
        else:
            current_chunk.append(line)
            current_size += line_size
    
    # Add remaining content
    if current_chunk:
        chunks.append('\n'.join(current_chunk))
    
    return chunks


print("✅ Helper classes and functions loaded!")

✅ Helper classes and functions loaded!


---

# Step 1: Initial Outline Generation

Configure an agent to generate a comprehensive content outline based on the specified topic.

In [6]:
import traceback

# Specify the subject matter for document creation
CONTENT_TOPIC = "AWS Lambda Best Practices and Production Deployment"
print(f"✓ CONTENT_TOPIC set to: {CONTENT_TOPIC}")

# Set up the outline creation agent with Bedrock
try:
    print("✓ Starting BedrockAgent initialization...")
    outline_creator = BedrockAgent(
        name="OutlineCreator",
        system_prompt="""You are a seasoned technical writer with expertise in AWS services.
        Create a thorough outline that covers:
        - Primary sections (5-7 core areas)
        - Supporting points per section (3-5 items each)
        - Smooth transitions connecting sections
        - Real-world examples and implementation scenarios

        Structure the outline using clear section labels and itemized lists."""
    )
    print("✓ BedrockAgent successfully initialized")
    
    # Generate the outline
    print("\n📝 Generating content outline...")
    outline = outline_creator.generate(
        f"Create a comprehensive outline for a technical guide on: {CONTENT_TOPIC}",
        temperature=0.7
    )
    
    print("\n" + "="*80)
    print("GENERATED OUTLINE:")
    print("="*80)
    print(outline)
    print("\n✅ Step 1 Complete: Outline generated!")
    
except Exception as e:
    print(f"✗ ERROR at BedrockAgent initialization or outline generation:")
    print(f"  Error type: {type(e).__name__}")
    print(f"  Error message: {str(e)}")
    traceback.print_exc()



✓ CONTENT_TOPIC set to: AWS Lambda Best Practices and Production Deployment
✓ Starting BedrockAgent initialization...
✓ BedrockAgent successfully initialized

📝 Generating content outline...

GENERATED OUTLINE:
I. Introduction to AWS Lambda
   A. Overview of serverless computing and AWS Lambda
   B. Benefits of using AWS Lambda
   C. Use cases for AWS Lambda

II. Best Practices for AWS Lambda Development
   A. Designing Lambda functions
      1. Stateless and idempotent functions
      2. Separation of concerns
      3. Modular and reusable code
   B. Writing efficient Lambda code
      1. Optimizing memory usage
      2. Minimizing execution time
      3. Handling asynchronous operations
   C. Testing and debugging Lambda functions
      1. Unit testing
      2. Integration testing
      3. Logging and monitoring

III. AWS Lambda Security Considerations
   A. IAM roles and permissions
   B. Data encryption at rest and in transit
   C. VPC configuration for Lambda functions
   D. Secur

In [7]:
import boto3
from botocore.exceptions import ClientError

# First, verify your credentials are loaded correctly
print("Step 1: Checking AWS credentials...\n")

try:
    session = boto3.Session()
    credentials = session.get_credentials()
    
    if credentials is None:
        print("✗ NO CREDENTIALS FOUND")
        print("  You need to configure AWS credentials. Options:")
        print("  1. Set environment variables: AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY")
        print("  2. Create ~/.aws/credentials file")
        print("  3. Use AWS CLI: aws configure")
        print("\n⚠️  Please configure credentials before proceeding.")
    else:
        print(f"✓ Credentials found")
        print(f"  Access Key: {credentials.access_key[:10]}...")
        print(f"  Secret Key: {credentials.secret_key[:10]}...")
        
        # Now test if they're valid
        print("\nStep 2: Testing credentials validity...\n")
        sts = session.client('sts', region_name='us-east-1')
        identity = sts.get_caller_identity()
        print(f"✓ Credentials are VALID")
        print(f"  Account ID: {identity['Account']}")
        print(f"  User ARN: {identity['Arn']}")
    
except ClientError as e:
    print(f"✗ Credentials are INVALID")
    print(f"  Error: {e}")
    print("\n  SOLUTION:")
    print("  1. Go to: https://console.aws.amazon.com/iam/")
    print("  2. Find your user and click 'Security credentials'")
    print("  3. Delete old Access Keys, create NEW ones")
    print("  4. Update your local credentials file or environment variables")
except Exception as e:
    print(f"✗ ERROR: {type(e).__name__}: {e}")
    print("⚠️  Please fix the error before proceeding with content generation.")

Step 1: Checking AWS credentials...

✓ Credentials found
  Access Key: AKIAQPRFFU...
  Secret Key: X7Kf/XK/H5...

Step 2: Testing credentials validity...

✓ Credentials are VALID
  Account ID: 033363961390
  User ARN: arn:aws:iam::033363961390:user/KiranFirstUser


---

# Step 2: Outline Evaluation

Assess the outline for completeness and logical structure using evaluation criteria.

In [8]:
# Define evaluation criteria
evaluation_criteria = {
    "completeness": "Outline covers all major aspects of the topic",
    "structure": "Logical organization with clear hierarchy",
    "depth": "Appropriate level of detail for each section",
    "coherence": "Sections flow logically from one to another",
    "actionability": "Includes practical, implementable guidance"
}

# Initialize evaluator
evaluator = OutlineEvaluator(criteria=evaluation_criteria)

# Evaluate the outline
print("🔍 Evaluating outline quality...\n")
try:
    evaluation_result = evaluator.evaluate(outline)
    
    print("="*80)
    print("OUTLINE EVALUATION RESULTS:")
    print("="*80)
    print(f"Score: {evaluation_result.score:.2f}/1.0")
    print(f"\nFeedback:\n{evaluation_result.feedback}")
    print("\n✅ Step 2 Complete: Outline evaluated!")
except NameError:
    print("⚠️ Error: 'outline' variable not found. Please run Step 1 first.")
except Exception as e:
    print(f"✗ Error during evaluation: {e}")
    traceback.print_exc()



🔍 Evaluating outline quality...

OUTLINE EVALUATION RESULTS:
Score: 0.90/1.0

Feedback:
The outline is comprehensive, well-structured, and covers all major aspects of AWS Lambda development and deployment. It starts with an introduction to serverless computing and AWS Lambda, followed by best practices for development, security considerations, performance optimization, and deployment strategies. The outline also includes integration with other AWS services, cost optimization strategies, and future considerations. The level of detail is appropriate for each section, and the sections flow logically from one to another. The outline includes practical guidance and actionable steps, making it useful for implementing AWS Lambda solutions in real-world scenarios.

✅ Step 2 Complete: Outline evaluated!


---

# Step 3: Coherence Enhancement

Analyze the outline structure and suggest improvements for better flow and organization.

In [9]:
# Configure coherence enhancement agent
coherence_agent = BedrockAgent(
    name="CoherenceEnhancer",
    system_prompt="""You are a document structure expert.
    Your task is to:
    1. Analyze the logical flow between sections
    2. Suggest transition statements between major topics
    3. Ensure progressive complexity (simple to advanced)
    4. Identify any gaps or redundancies
    5. Optimize the reading experience
    
    Provide the enhanced outline with smooth transitions."""
)

# Apply coherence enhancement
print("🔧 Enhancing outline coherence and flow...\n")
try:
    enhanced_outline = coherence_agent.generate(
        f"""Enhance the coherence and flow of this outline:
        
        {outline}
        
        Add transition notes between sections and ensure logical progression."""
    )
    
    print("="*80)
    print("ENHANCED OUTLINE WITH TRANSITIONS:")
    print("="*80)
    print(enhanced_outline)
    print("\n✅ Step 3 Complete: Coherence enhanced!")
except NameError:
    print("⚠️ Error: 'outline' variable not found. Please run Step 1 first.")
except Exception as e:
    print(f"✗ Error during coherence enhancement: {e}")
    traceback.print_exc()

🔧 Enhancing outline coherence and flow...

ENHANCED OUTLINE WITH TRANSITIONS:
Here's the enhanced outline with smooth transitions and a logical flow:

I. Introduction to AWS Lambda
   A. Overview of serverless computing and AWS Lambda
   B. Benefits of using AWS Lambda
   C. Use cases for AWS Lambda

II. Best Practices for AWS Lambda Development
   Transition: To effectively leverage AWS Lambda, it's crucial to follow best practices during development.
   A. Designing Lambda functions
      1. Stateless and idempotent functions
      2. Separation of concerns
      3. Modular and reusable code
   B. Writing efficient Lambda code
      Transition: In addition to good design principles, optimizing code efficiency is essential for serverless functions.
      1. Optimizing memory usage
      2. Minimizing execution time
      3. Handling asynchronous operations
   C. Testing and debugging Lambda functions
      Transition: Robust testing and debugging strategies are key to ensuring the rel

---

# Step 4: Section Development

Draft detailed content for each section while maintaining context and consistency.

In [10]:
# Configure content development agent
content_agent = BedrockAgent(
    name="ContentDeveloper",
    system_prompt="""You are an expert AWS technical writer.
    Develop detailed, practical content for each section that includes:
    - Clear explanations of concepts
    - Code examples where appropriate
    - Best practices and common pitfalls
    - Real-world use cases
    - Step-by-step instructions
    
    Write in a professional yet accessible tone for technical professionals."""
)

try:
    # Parse sections from the enhanced outline (or original outline if enhanced not available)
    outline_to_use = enhanced_outline if 'enhanced_outline' in globals() else outline
    sections = parse_sections(outline_to_use)
    print(f"📝 Developing content for {len(sections)} sections...\n")

    # Generate content for each section
    full_content = f"# {CONTENT_TOPIC}\n\n"
    full_content += f"*Generated on {datetime.now().strftime('%Y-%m-%d')}*\n\n"
    full_content += "---\n\n"

    for i, section in enumerate(sections[:5], 1):  # Limit to first 5 sections for demo
        print(f"Writing section {i}/{min(5, len(sections))}: {section}")
        
        section_content = content_agent.generate(
            f"""Write comprehensive content for this section:
            
            Section Title: {section}
            
            Context: This is part of a guide on {CONTENT_TOPIC}
            
            Include practical examples, code snippets, and actionable advice.
            Aim for 400-600 words.""",
            temperature=0.7
        )
        
        full_content += f"## {section}\n\n{section_content}\n\n---\n\n"

    print("\n" + "="*80)
    print("FULL CONTENT PREVIEW (First 1000 characters):")
    print("="*80)
    print(full_content[:1000] + "...")
    print("\n✅ Step 4 Complete: Section content developed!")
except NameError as e:
    print(f"⚠️ Error: Required variable not found ({e}). Please run previous steps first.")
except Exception as e:
    print(f"✗ Error during content development: {e}")
    traceback.print_exc()

📝 Developing content for 27 sections...

Writing section 1/5: Stateless and idempotent functions
Writing section 2/5: Separation of concerns
Writing section 3/5: Modular and reusable code
Writing section 4/5: Optimizing memory usage
Writing section 5/5: Minimizing execution time

FULL CONTENT PREVIEW (First 1000 characters):
# AWS Lambda Best Practices and Production Deployment

*Generated on 2025-12-05*

---

## Stateless and idempotent functions

Stateless and Idempotent Functions

In the context of AWS Lambda, it's crucial to design your functions to be stateless and idempotent. These characteristics ensure that your Lambda functions are reliable, scalable, and fault-tolerant, which are essential qualities for production deployments.

Stateless Functions:
A stateless function is one that does not maintain any internal state or memory between invocations. Each invocation of the function should be independent and self-contained, without relying on any data or state from previous invoc

---

# Step 5: Content Refinement

Apply style, tone, and readability improvements to the generated content.

In [11]:
# Configure style refinement agent
refiner_agent = BedrockAgent(
    name="StyleRefiner",
    system_prompt="""You are a professional technical editor.
    Refine content to ensure:
    - Active voice where possible
    - Varied sentence length for readability
    - Technical terms are defined on first use
    - Consistent terminology throughout
    - Professional yet accessible tone
    - Clear, concise explanations
    - Proper formatting and structure
    
    Maintain all technical accuracy while improving readability."""
)

# Apply refinement
print("✨ Refining content style and readability...\n")
try:
    refined_content = refiner_agent.generate(
        f"""Refine this technical content for maximum clarity and professionalism:
        
        Target audience: Technical professionals and DevOps engineers
        Purpose: Educational guide
        
        Content:
        {full_content}
        
        Improve style, tone, and readability while preserving all technical details.""",
        max_tokens=8000
    )
    
    print("="*80)
    print("REFINED CONTENT PREVIEW (First 1000 characters):")
    print("="*80)
    print(refined_content[:1000] + "...")
    print("\n✅ Step 5 Complete: Content refined!")
except NameError:
    print("⚠️ Error: 'full_content' variable not found. Please run Step 4 first.")
except Exception as e:
    print(f"✗ Error during content refinement: {e}")
    traceback.print_exc()

✨ Refining content style and readability...

REFINED CONTENT PREVIEW (First 1000 characters):
Here is the content with refined style, tone, and readability while preserving all technical details:

# AWS Lambda Best Practices and Production Deployment

*Updated December 2025*

## Stateless and Idempotent Functions

In serverless architectures using AWS Lambda, it's crucial to design functions that are stateless and idempotent. These characteristics ensure reliability, scalability, and fault tolerance - essential qualities for production deployments.

### Stateless Functions

A stateless function does not maintain internal state or memory between invocations. Each invocation operates independently without relying on data from previous ones. This approach simplifies logic, enhances scalability, and improves fault tolerance.  

**Benefits of Stateless Functions:**

1. **Scalability:** Stateless functions can scale horizontally by running multiple concurrent instances, as each operates inde

---

# Step 6: Quality Assurance

Identify and correct linguistic errors, ensuring professional-quality output.

In [12]:
# Configure QA agent
qa_agent = BedrockAgent(
    name="QualityAssurance",
    system_prompt="""You are a meticulous technical editor and QA specialist.
    Review content for:
    - Grammar and spelling errors
    - Punctuation and formatting issues
    - Consistency in terminology
    - Technical accuracy
    - Clarity and readability
    - Proper code formatting
    - Link validity and references
    
    Correct any errors while maintaining the author's voice and technical accuracy."""
)

# Perform quality checks
print("🔍 Performing quality assurance checks...\n")
try:
    # Use refined_content if available, otherwise use full_content
    content_to_qa = refined_content if 'refined_content' in globals() else full_content
    
    final_content = qa_agent.generate(
        f"""Perform comprehensive quality assurance on this content:
        
        {content_to_qa}
        
        Correct any errors and ensure professional quality.
        Return the corrected version.""",
        max_tokens=8000
    )
    
    print("="*80)
    print("FINAL CONTENT PREVIEW (First 1500 characters):")
    print("="*80)
    print(final_content[:1500] + "...")
    print("\n✅ Step 6 Complete: Quality assurance passed!")
except NameError:
    print("⚠️ Error: Required content variable not found. Please run previous steps first.")
except Exception as e:
    print(f"✗ Error during quality assurance: {e}")
    traceback.print_exc()

🔍 Performing quality assurance checks...

FINAL CONTENT PREVIEW (First 1500 characters):
Here is the content with corrections and quality assurance:

# AWS Lambda Best Practices and Production Deployment

*Updated December 2025*

## Stateless and Idempotent Functions

In serverless architectures using AWS Lambda, it's crucial to design functions that are stateless and idempotent. These characteristics ensure reliability, scalability, and fault tolerance - essential qualities for production deployments.

### Stateless Functions

A stateless function does not maintain internal state or memory between invocations. Each invocation operates independently without relying on data from previous ones. This approach simplifies logic, enhances scalability, and improves fault tolerance.

**Benefits of Stateless Functions:**

1. **Scalability:** Stateless functions can scale horizontally by running multiple concurrent instances, as each operates independently.
2. **Fault Tolerance:** If an instance

---

# Step 7: Final Output & Export

Produce the final polished content and export to file.

In [13]:
import os

try:
    # Check if final_content exists, otherwise use available content
    if 'final_content' not in globals():
        if 'refined_content' in globals():
            final_content = refined_content
        elif 'full_content' in globals():
            final_content = full_content
        else:
            raise NameError("No content available to export. Please run content generation steps first.")
    
    # Create output directory
    output_dir = "./generated_content"
    os.makedirs(output_dir, exist_ok=True)

    # Generate filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"aws_lambda_guide_{timestamp}.md"
    output_path = os.path.join(output_dir, filename)

    # Add metadata header
    metadata = f"""---
title: {CONTENT_TOPIC}
author: Strands Agent System (AWS Bedrock)
date: {datetime.now().strftime('%Y-%m-%d')}
generated_by: Multi-Agent Content Generation Pipeline
model: Claude 3 Sonnet (AWS Bedrock)
---

"""

    final_document = metadata + final_content

    # Write to file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(final_document)

    print("="*80)
    print("📄 FINAL DOCUMENT GENERATED SUCCESSFULLY!")
    print("="*80)
    print(f"File: {output_path}")
    print(f"Size: {len(final_document)} characters")
    print(f"Word count: ~{len(final_document.split())} words")
    print("\n✅ Step 7 Complete: Document exported!")

    # Display full content preview
    print("\n" + "="*80)
    print("COMPLETE FINAL DOCUMENT PREVIEW (First 2000 characters):")
    print("="*80)
    print(final_document[:2000] + "...")
    
except NameError as e:
    print(f"⚠️ Error: {e}")
except Exception as e:
    print(f"✗ Error during document export: {e}")
    traceback.print_exc()

📄 FINAL DOCUMENT GENERATED SUCCESSFULLY!
File: ./generated_content\aws_lambda_guide_20251205_171732.md
Size: 19336 characters
Word count: ~2550 words

✅ Step 7 Complete: Document exported!

COMPLETE FINAL DOCUMENT PREVIEW (First 2000 characters):
---
title: AWS Lambda Best Practices and Production Deployment
author: Strands Agent System (AWS Bedrock)
date: 2025-12-05
generated_by: Multi-Agent Content Generation Pipeline
model: Claude 3 Sonnet (AWS Bedrock)
---

Here is the content with corrections and quality assurance:

# AWS Lambda Best Practices and Production Deployment

*Updated December 2025*

## Stateless and Idempotent Functions

In serverless architectures using AWS Lambda, it's crucial to design functions that are stateless and idempotent. These characteristics ensure reliability, scalability, and fault tolerance - essential qualities for production deployments.

### Stateless Functions

A stateless function does not maintain internal state or memory between invocations. Each

---

# Analytics & Insights

Review the agent performance and conversation history.

In [14]:
# Collect analytics from all agents
try:
    agents = []
    if 'outline_creator' in globals():
        agents.append(outline_creator)
    if 'coherence_agent' in globals():
        agents.append(coherence_agent)
    if 'content_agent' in globals():
        agents.append(content_agent)
    if 'refiner_agent' in globals():
        agents.append(refiner_agent)
    if 'qa_agent' in globals():
        agents.append(qa_agent)
    
    if not agents:
        print("⚠️ No agents found. Please run the previous steps first.")
    else:
        print("="*80)
        print("AGENT PERFORMANCE ANALYTICS")
        print("="*80)

        total_interactions = 0
        for agent in agents:
            history = agent.get_history()
            total_interactions += len(history)
            print(f"\n{agent.name}:")
            print(f"  - Interactions: {len(history)}")
            print(f"  - Model: {agent.model_id}")

        print(f"\n{'='*80}")
        print(f"Total Agent Interactions: {total_interactions}")
        
        if 'final_document' in globals():
            print(f"Final Document Length: {len(final_document)} characters")
            print(f"Estimated Word Count: {len(final_document.split())} words")
        elif 'final_content' in globals():
            print(f"Final Content Length: {len(final_content)} characters")
            print(f"Estimated Word Count: {len(final_content.split())} words")
        
        print(f"\n✅ Analytics complete!")
        
except Exception as e:
    print(f"✗ Error during analytics collection: {e}")
    traceback.print_exc()

AGENT PERFORMANCE ANALYTICS

OutlineCreator:
  - Interactions: 1
  - Model: anthropic.claude-3-sonnet-20240229-v1:0

CoherenceEnhancer:
  - Interactions: 1
  - Model: anthropic.claude-3-sonnet-20240229-v1:0

ContentDeveloper:
  - Interactions: 5
  - Model: anthropic.claude-3-sonnet-20240229-v1:0

StyleRefiner:
  - Interactions: 1
  - Model: anthropic.claude-3-sonnet-20240229-v1:0

QualityAssurance:
  - Interactions: 1
  - Model: anthropic.claude-3-sonnet-20240229-v1:0

Total Agent Interactions: 9
Final Document Length: 19336 characters
Estimated Word Count: 2550 words

✅ Analytics complete!


---

# Production Deployment Considerations

## Key Implementation Notes:

### 1. **Error Handling & Retries**
```python
# Add retry logic for API calls
from botocore.exceptions import ClientError
import time

def invoke_with_retry(bedrock_client, model_id, body, max_retries=3):
    for attempt in range(max_retries):
        try:
            return bedrock_client.invoke_model(modelId=model_id, body=body)
        except ClientError as e:
            if attempt == max_retries - 1:
                raise
            time.sleep(2 ** attempt)  # Exponential backoff
```

### 2. **Cost Optimization**
- Monitor token usage per request
- Use appropriate model sizes (Haiku for simple tasks, Sonnet for complex)
- Implement caching for repeated queries
- Set max_tokens limits appropriately

### 3. **Observability**
- Log all agent interactions to CloudWatch
- Track latency and token usage metrics
- Implement distributed tracing with X-Ray
- Monitor error rates and retry patterns

### 4. **Security**
- Use IAM roles instead of access keys in production
- Implement least-privilege permissions
- Encrypt sensitive data at rest and in transit
- Use AWS Secrets Manager for credentials

### 5. **Scalability**
- Implement async processing for multiple documents
- Use SQS for job queuing
- Deploy as Lambda functions for serverless scaling
- Consider Step Functions for complex workflows

### 6. **Testing Strategy**
- Unit tests for each agent component
- Integration tests for the full pipeline
- Quality benchmarks for output evaluation
- A/B testing for prompt optimization

---

## Next Steps for Production:

1. **Containerize** the application with Docker
2. **Deploy** to AWS ECS or Lambda
3. **Set up CI/CD** pipeline with CodePipeline
4. **Implement monitoring** with CloudWatch and X-Ray
5. **Add API Gateway** for external access
6. **Configure auto-scaling** based on demand
7. **Implement content versioning** with S3
8. **Add human-in-the-loop** review workflow

---

# 🎉 Congratulations!

You've successfully built and executed a complete multi-agent content generation system using:
- ✅ AWS Bedrock for LLM capabilities
- ✅ Multi-agent architecture with specialized roles
- ✅ Quality evaluation and iterative improvement
- ✅ Production-ready error handling
- ✅ Comprehensive analytics and monitoring

## What You've Learned:
1. How to configure AWS Bedrock clients
2. Building specialized agents with custom prompts
3. Implementing evaluation and feedback loops
4. Multi-stage content refinement pipeline
5. Production deployment considerations

## Customize This System:
- Change the `TOPIC` variable to generate content on any subject
- Modify agent prompts for different writing styles
- Add more evaluation criteria
- Integrate with your own data sources
- Deploy to production AWS infrastructure

---

*Generated by Strands Agent System powered by AWS Bedrock*